# Melon 주간 차트 크롤링

In [5]:
import requests
import pandas as pd

In [6]:
# 멜론 주간차트
# 헤더에 User-Agent 값을 주어야만 데이터를 가져올 수 있음.
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
req = requests.get(url, headers=header)
html = req.text
#html

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [9]:
trs = soup.select('#lst50')
len(trs)

50

In [10]:
tr = trs[0]
tr

<tr class="lst50" data-song-no="33507137" id="lst50">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="Butter 곡 선택" type="checkbox" value="33507137"/></div></td>
<td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="순위 동일">
<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
<span class="none">0</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('10612483');" title="Butter">
<img alt="Butter - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize" width="60">
<span class="bg_album_frame"></span>
</img></a>
</div></td>
<td><div class="wrap">
<a class="btn button_icons type03 song_info" href="javascript:melon.link.go

In [11]:
# 순위
rank = tr.select_one('.rank').string    # get_text()
rank = int(rank)
rank

1

In [12]:
# 곡명
title = tr.select_one('.ellipsis.rank01')
title = title.find('a').get_text()
title

'Butter'

In [13]:
# 가수
artist = tr.select_one('.ellipsis.rank02')
artist = artist.find('a').get_text()
artist

'방탄소년단'

In [14]:
# 가수
artist = tr.select_one('.ellipsis.rank02')
artist = artist.find('a').get_text()
artist

'방탄소년단'

In [15]:
# 앨범
album = tr.select_one('.ellipsis.rank03')
album = album.find('a').string
album

'Butter'

- 50곡에 대해서 데이터프레임 만들기

In [16]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.rank').string    # get_text()
    rank = int(rank)
    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()
    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [17]:
df = pd.DataFrame({
    '순위': rank_list,
    '곡명': title_list,
    '가수': artist_list,
    '앨범': album_list
})
df.head()

,순위,곡명,가수,앨범
0,1,Butter,방탄소년단,Butter
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL
2,3,헤픈 우연,헤이즈 (Heize),HAPPEN
3,4,Next Level,aespa,Next Level
4,5,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice


- 이미지 속성값 가져오기

In [18]:
image = trs[0].select_one('.image_typeAll').find('img')
image = image['src']
image

'https://cdnimg.melon.co.kr/cm2/album/images/106/12/483/10612483_20210521111412_500.jpg/melon/resize/120/quality/80/optimize'

- 100 곡 모두 가져오기

In [19]:
trs = soup.select('#lst50')
trs2 = soup.select('#lst100')
trs.extend(trs2)
len(trs)

100

In [20]:
rank_list, title_list, artist_list, album_list, image_list = [],[],[],[],[]
for tr in trs:
    rank = tr.select_one('.rank').string    # get_text()
    rank = int(rank)
    title = tr.select_one('.ellipsis.rank01')
    title = title.find('a').get_text()
    artist = tr.select_one('.ellipsis.rank02')
    artist = artist.find('a').get_text()
    album = tr.select_one('.ellipsis.rank03')
    album = album.find('a').string
    image = tr.select_one('.image_typeAll').find('img')
    image = image['src']

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    image_list.append(image)

In [21]:
df = pd.DataFrame({
    '순위': rank_list,
    '곡명': title_list,
    '가수': artist_list,
    '앨범': album_list,
    '이미지': image_list
})
df.head()

,순위,곡명,가수,앨범,이미지
0,1,Butter,방탄소년단,Butter,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,2,Dun Dun Dance,오마이걸 (OH MY GIRL),Dear OHMYGIRL,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,3,헤픈 우연,헤이즈 (Heize),HAPPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,4,Next Level,aespa,Next Level,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,5,Peaches (Feat. Daniel Caesar & Giveon),Justin Bieber,Justice,https://cdnimg.melon.co.kr/cm2/album/images/10...


- 파일에 저장하기

In [22]:
period = soup.select_one('.yyyymmdd').get_text().strip()
period

'2021.05.31 ~ 2021.06.06'

In [23]:
filename = f'melon({period}).csv'
df.to_csv(filename, index=False, sep=',', encoding='utf8')